In [1]:
!apt-get update
!apt-get install graphviz -y

!pip install tensorflow
!pip install numpy
!pip install pandas

!pip install keras
!pip install scikit-learn
!pip install matplotlib
!pip install joblib
!pip install pyarrow
!pip install fastparquet
!pip install scipy
!pip install seaborn
!pip install tqdm
!pip install pydot
!pip install tensorflow-io
!pip install tensorflow-addons

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (278 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 120 not upgraded.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run

In [2]:
import tensorflow as tf
import keras

print(f"Keras version: {keras.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"CUDA available: {tf.test.is_built_with_cuda()}")
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")

# GPU configuration
import tensorflow as tf
import os

# Limita la crescita della memoria GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Imposta la crescita di memoria dinamica
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        # Opzionalmente, limita la memoria GPU massima (uncomment se necessario)
        # tf.config.experimental.set_virtual_device_configuration(
        #     gpus[0],
        #     [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*4)]  # 4GB
        # )
        
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
        
# Imposta le opzioni di logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Riduce i messaggi di log
        
# Configura la modalità mista di precisione
tf.keras.mixed_precision.set_global_policy('float32')

# Imposta il seed per la riproducibilità
##tf.random.set_seed(42)

2024-11-12 13:23:10.750385: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 13:23:10.750440: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 13:23:10.750502: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 13:23:10.761720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Keras version: 2.14.0
TensorFlow version: 2.14.0
TensorFlow version: 2.14.0
CUDA available: True
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


2024-11-12 13:23:13.297044: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43404 MB memory:  -> device: 0, name: NVIDIA L40, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa
from datetime import datetime
import os
import joblib
import re
from typing import List

random_state_value = None
execute_name = datetime.now().strftime("%Y-%m-%d_%H-%M")

base_project_dir = './'
data_dir = '../../sources/'
models_project_dir = base_project_dir

os.makedirs(base_project_dir, exist_ok=True)
os.makedirs(models_project_dir, exist_ok=True)

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
def clean_column_name(name: str) -> str:
    """
    Rimuove caratteri speciali e spazi, converte in snake_case e abbrevia.

    Parameters
    ----------
    name : str
        Nome della colonna da pulire

    Returns
    -------
    str
        Nome della colonna pulito
    """
    # Rimuove caratteri speciali
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)
    # Converte in snake_case
    name = name.lower().replace(' ', '_')

    # Abbreviazioni comuni
    abbreviations = {
        'production': 'prod',
        'percentage': 'pct',
        'hectare': 'ha',
        'tonnes': 't',
        'litres': 'l',
        'minimum': 'min',
        'maximum': 'max',
        'average': 'avg'
    }

    for full, abbr in abbreviations.items():
        name = name.replace(full, abbr)

    return name


def clean_column_names(df: pd.DataFrame) -> List[str]:
    """
    Pulisce tutti i nomi delle colonne in un DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con le colonne da pulire

    Returns
    -------
    list
        Lista dei nuovi nomi delle colonne puliti
    """
    new_columns = []

    for col in df.columns:
        # Usa regex per separare le varietà
        varieties = re.findall(r'([a-z]+)_([a-z_]+)', col)
        if varieties:
            new_columns.append(f"{varieties[0][0]}_{varieties[0][1]}")
        else:
            new_columns.append(col)

    return new_columns

In [5]:
def save_plot(plt, title, output_dir=f'{base_project_dir}/{execute_name}_plots'):
    os.makedirs(output_dir, exist_ok=True)
    filename = "".join(x for x in title if x.isalnum() or x in [' ', '-', '_']).rstrip()
    filename = filename.replace(' ', '_').lower()
    filepath = os.path.join(output_dir, f"{filename}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Plot salvato come: {filepath}")


def encode_techniques(df, mapping_path=f'{data_dir}technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}. Run create_technique_mapping first.")

    technique_mapping = joblib.load(mapping_path)

    # Trova tutte le colonne delle tecniche
    tech_columns = [col for col in df.columns if col.endswith('_tech')]

    # Applica il mapping a tutte le colonne delle tecniche
    for col in tech_columns:
        df[col] = df[col].str.lower().map(technique_mapping).fillna(0).astype(int)

    return df


def decode_single_technique(technique_value, mapping_path=f'{data_dir}technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}")

    technique_mapping = joblib.load(mapping_path)
    reverse_mapping = {v: k for k, v in technique_mapping.items()}
    reverse_mapping[0] = ''

    return reverse_mapping.get(technique_value, '')


def prepare_comparison_data(simulated_data, olive_varieties):
    # Pulisci i nomi delle colonne
    df = simulated_data.copy()

    df.columns = clean_column_names(df)
    df = encode_techniques(df)

    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]
    comparison_data = []

    for variety in varieties:
        olive_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_olive_prod')), None)
        oil_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_avg_oil_prod')), None)
        tech_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_tech')), None)
        water_need_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_water_need')), None)

        if olive_prod_col and oil_prod_col and tech_col and water_need_col:
            variety_data = df[[olive_prod_col, oil_prod_col, tech_col, water_need_col]]
            variety_data = variety_data[variety_data[tech_col] != 0]  # Esclude le righe dove la tecnica è 0

            if not variety_data.empty:
                avg_olive_prod = pd.to_numeric(variety_data[olive_prod_col], errors='coerce').mean()
                avg_oil_prod = pd.to_numeric(variety_data[oil_prod_col], errors='coerce').mean()
                avg_water_need = pd.to_numeric(variety_data[water_need_col], errors='coerce').mean()
                efficiency = avg_oil_prod / avg_olive_prod if avg_olive_prod > 0 else 0
                water_efficiency = avg_oil_prod / avg_water_need if avg_water_need > 0 else 0

                comparison_data.append({
                    'Variety': variety,
                    'Avg Olive Production (kg/ha)': avg_olive_prod,
                    'Avg Oil Production (L/ha)': avg_oil_prod,
                    'Avg Water Need (m³/ha)': avg_water_need,
                    'Oil Efficiency (L/kg)': efficiency,
                    'Water Efficiency (L oil/m³ water)': water_efficiency
                })

    return pd.DataFrame(comparison_data)


def plot_variety_comparison(comparison_data, metric):
    plt.figure(figsize=(12, 6))
    bars = plt.bar(comparison_data['Variety'], comparison_data[metric])
    plt.title(f'Comparison of {metric} across Olive Varieties')
    plt.xlabel('Variety')
    plt.ylabel(metric)
    plt.xticks(rotation=45, ha='right')

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{height:.2f}',
                 ha='center', va='bottom')

    plt.tight_layout()
    plt.show()
    save_plot(plt, f'variety_comparison_{metric.lower().replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "")}')
    plt.close()


def plot_efficiency_vs_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Olive Production (kg/ha)'],
                comparison_data['Oil Efficiency (L/kg)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Olive Production (kg/ha)'], row['Oil Efficiency (L/kg)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Oil Efficiency vs Olive Production by Variety')
    plt.xlabel('Average Olive Production (kg/ha)')
    plt.ylabel('Oil Efficiency (L oil / kg olives)')
    plt.tight_layout()
    save_plot(plt, 'efficiency_vs_production')
    plt.close()


def plot_water_efficiency_vs_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Olive Production (kg/ha)'],
                comparison_data['Water Efficiency (L oil/m³ water)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Olive Production (kg/ha)'], row['Water Efficiency (L oil/m³ water)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Water Efficiency vs Olive Production by Variety')
    plt.xlabel('Average Olive Production (kg/ha)')
    plt.ylabel('Water Efficiency (L oil / m³ water)')
    plt.tight_layout()
    plt.show()
    save_plot(plt, 'water_efficiency_vs_production')
    plt.close()


def plot_water_need_vs_oil_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Water Need (m³/ha)'],
                comparison_data['Avg Oil Production (L/ha)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Water Need (m³/ha)'], row['Avg Oil Production (L/ha)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Oil Production vs Water Need by Variety')
    plt.xlabel('Average Water Need (m³/ha)')
    plt.ylabel('Average Oil Production (L/ha)')
    plt.tight_layout()
    plt.show()
    save_plot(plt, 'water_need_vs_oil_production')
    plt.close()


def analyze_by_technique(simulated_data, olive_varieties):
    # Pulisci i nomi delle colonne
    df = simulated_data.copy()

    df.columns = clean_column_names(df)
    df = encode_techniques(df)
    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]

    technique_data = []

    for variety in varieties:
        olive_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_olive_prod')), None)
        oil_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_avg_oil_prod')), None)
        tech_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_tech')), None)
        water_need_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_water_need')), None)

        if olive_prod_col and oil_prod_col and tech_col and water_need_col:
            variety_data = df[[olive_prod_col, oil_prod_col, tech_col, water_need_col]]
            variety_data = variety_data[variety_data[tech_col] != 0]

            if not variety_data.empty:
                for tech in variety_data[tech_col].unique():
                    tech_data = variety_data[variety_data[tech_col] == tech]

                    avg_olive_prod = pd.to_numeric(tech_data[olive_prod_col], errors='coerce').mean()
                    avg_oil_prod = pd.to_numeric(tech_data[oil_prod_col], errors='coerce').mean()
                    avg_water_need = pd.to_numeric(tech_data[water_need_col], errors='coerce').mean()

                    efficiency = avg_oil_prod / avg_olive_prod if avg_olive_prod > 0 else 0
                    water_efficiency = avg_oil_prod / avg_water_need if avg_water_need > 0 else 0

                    technique_data.append({
                        'Variety': variety,
                        'Technique': tech,
                        'Technique String': decode_single_technique(tech),
                        'Avg Olive Production (kg/ha)': avg_olive_prod,
                        'Avg Oil Production (L/ha)': avg_oil_prod,
                        'Avg Water Need (m³/ha)': avg_water_need,
                        'Oil Efficiency (L/kg)': efficiency,
                        'Water Efficiency (L oil/m³ water)': water_efficiency
                    })

    return pd.DataFrame(technique_data)

In [6]:
def calculate_real_error(model, test_data, test_targets, scaler_y):
    # Fare predizioni
    predictions = model.predict(test_data)

    # Denormalizzare predizioni e target
    predictions_real = scaler_y.inverse_transform(predictions)
    targets_real = scaler_y.inverse_transform(test_targets)

    # Calcolare errore percentuale per ogni target
    percentage_errors = []
    absolute_errors = []

    for i in range(predictions_real.shape[1]):
        mae = np.mean(np.abs(predictions_real[:, i] - targets_real[:, i]))
        mape = np.mean(np.abs((predictions_real[:, i] - targets_real[:, i]) / targets_real[:, i])) * 100
        percentage_errors.append(mape)
        absolute_errors.append(mae)

    # Stampa risultati per ogni target
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']

    print("\nErrori per target:")
    print("-" * 50)
    for i, target in enumerate(target_names):
        print(f"{target}:")
        print(f"MAE assoluto: {absolute_errors[i]:.2f}")
        print(f"Errore percentuale medio: {percentage_errors[i]:.2f}%")
        print(f"Precisione: {100 - percentage_errors[i]:.2f}%")
        print("-" * 50)

    return percentage_errors, absolute_errors

In [7]:
simulated_data = pd.read_parquet(f"{data_dir}simulated_data.parquet")
olive_varieties = pd.read_parquet(f"{data_dir}olive_varieties.parquet")
# Esecuzione dell'analisi
comparison_data = prepare_comparison_data(simulated_data, olive_varieties)

# Genera i grafici
plot_variety_comparison(comparison_data, 'Avg Olive Production (kg/ha)')
plot_variety_comparison(comparison_data, 'Avg Oil Production (L/ha)')
plot_variety_comparison(comparison_data, 'Avg Water Need (m³/ha)')
plot_variety_comparison(comparison_data, 'Oil Efficiency (L/kg)')
plot_variety_comparison(comparison_data, 'Water Efficiency (L oil/m³ water)')
plot_efficiency_vs_production(comparison_data)
plot_water_efficiency_vs_production(comparison_data)
plot_water_need_vs_oil_production(comparison_data)

# Analisi per tecnica
technique_data = analyze_by_technique(simulated_data, olive_varieties)

print(technique_data)

# Stampa un sommario statistico
print("Comparison by Variety:")
print(comparison_data.set_index('Variety'))
print("\nBest Varieties by Water Efficiency:")
print(comparison_data.sort_values('Water Efficiency (L oil/m³ water)', ascending=False).head())

KeyError: 'Variety'

<Figure size 1200x600 with 0 Axes>

In [ ]:
def prepare_transformer_data(df, olive_varieties_df):
    # Crea una copia del DataFrame per evitare modifiche all'originale
    df = df.copy()

    # Ordina per zona e anno
    df = df.sort_values(['zone', 'year'])

    # Definisci le feature
    temporal_features = ['temp_mean', 'precip_sum', 'solar_energy_sum']
    static_features = ['ha']  # Feature statiche base
    target_features = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']

    # Ottieni le varietà pulite
    all_varieties = olive_varieties_df['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]

    # Crea la struttura delle feature per ogni varietà
    variety_features = [
        'tech', 'pct', 'prod_t_ha', 'oil_prod_t_ha', 'oil_prod_l_ha',
        'min_yield_pct', 'max_yield_pct', 'min_oil_prod_l_ha', 'max_oil_prod_l_ha',
        'avg_oil_prod_l_ha', 'l_per_t', 'min_l_per_t', 'max_l_per_t', 'avg_l_per_t'
    ]

    # Prepara dizionari per le nuove colonne
    new_columns = {}

    # Prepara le feature per ogni varietà
    for variety in varieties:
        # Feature esistenti
        for feature in variety_features:
            col_name = f"{variety}_{feature}"
            if col_name in df.columns:
                if feature != 'tech':  # Non includere la colonna tech direttamente
                    static_features.append(col_name)

        # Feature binarie per le tecniche di coltivazione
        for technique in ['tradizionale', 'intensiva', 'superintensiva']:
            col_name = f"{variety}_{technique}"
            new_columns[col_name] = df[f"{variety}_tech"].notna() & (
                    df[f"{variety}_tech"].str.lower() == technique
            ).fillna(False)
            static_features.append(col_name)

    # Aggiungi tutte le nuove colonne in una volta sola
    new_df = pd.concat([df] + [pd.Series(v, name=k) for k, v in new_columns.items()], axis=1)

    # Ordiniamo per zona e anno per mantenere la continuità temporale
    df_sorted = new_df.sort_values(['zone', 'year'])

    # Definiamo la dimensione della finestra temporale
    window_size = 41

    # Liste per raccogliere i dati
    temporal_sequences = []
    static_features_list = []
    targets_list = []

    # Iteriamo per ogni zona
    for zone in df_sorted['zone'].unique():
        zone_data = df_sorted[df_sorted['zone'] == zone].reset_index(drop=True)

        if len(zone_data) >= window_size:  # Verifichiamo che ci siano abbastanza dati
            # Creiamo sequenze temporali scorrevoli
            for i in range(len(zone_data) - window_size + 1):
                # Sequenza temporale
                temporal_window = zone_data.iloc[i:i + window_size][temporal_features].values
                # Verifichiamo che non ci siano valori NaN
                if not np.isnan(temporal_window).any():
                    temporal_sequences.append(temporal_window)

                    # Feature statiche (prendiamo quelle dell'ultimo timestep della finestra)
                    static_features_list.append(zone_data.iloc[i + window_size - 1][static_features].values)

                    # Target (prendiamo quelli dell'ultimo timestep della finestra)
                    targets_list.append(zone_data.iloc[i + window_size - 1][target_features].values)

    # Convertiamo in array numpy
    X_temporal = np.array(temporal_sequences)
    X_static = np.array(static_features_list)
    y = np.array(targets_list)

    print(f"Dataset completo - Temporal: {X_temporal.shape}, Static: {X_static.shape}, Target: {y.shape}")

    # Split dei dati (usando indici casuali per una migliore distribuzione)
    indices = np.random.permutation(len(X_temporal))

    #train_idx = int(len(indices) * 0.7)        # 70% training
    #val_idx = int(len(indices) * 0.85)         # 15% validation
    # Il resto rimane 15% test

    train_idx = int(len(indices) * 0.65)        # 65% training
    val_idx = int(len(indices) * 0.85)          # 20% validation
    # Il resto rimane 15% test

    #train_idx = int(len(indices) * 0.60)       # 60% training
    #val_idx = int(len(indices) * 0.85)         # 25% validation
    # Il resto rimane 15% test

    train_indices = indices[:train_idx]
    val_indices = indices[train_idx:val_idx]
    test_indices = indices[val_idx:]

    # Split dei dati
    X_temporal_train = X_temporal[train_indices]
    X_temporal_val = X_temporal[val_indices]
    X_temporal_test = X_temporal[test_indices]

    X_static_train = X_static[train_indices]
    X_static_val = X_static[val_indices]
    X_static_test = X_static[test_indices]

    y_train = y[train_indices]
    y_val = y[val_indices]
    y_test = y[test_indices]

    # Standardizzazione
    scaler_temporal = StandardScaler()
    scaler_static = StandardScaler()
    scaler_y = StandardScaler()

    # Standardizzazione dei dati temporali
    X_temporal_train = scaler_temporal.fit_transform(X_temporal_train.reshape(-1, len(temporal_features))).reshape(X_temporal_train.shape)
    X_temporal_val = scaler_temporal.transform(X_temporal_val.reshape(-1, len(temporal_features))).reshape(X_temporal_val.shape)
    X_temporal_test = scaler_temporal.transform(X_temporal_test.reshape(-1, len(temporal_features))).reshape(X_temporal_test.shape)

    # Standardizzazione dei dati statici
    X_static_train = scaler_static.fit_transform(X_static_train)
    X_static_val = scaler_static.transform(X_static_val)
    X_static_test = scaler_static.transform(X_static_test)

    # Standardizzazione dei target
    y_train = scaler_y.fit_transform(y_train)
    y_val = scaler_y.transform(y_val)
    y_test = scaler_y.transform(y_test)

    print("\nShape dopo lo split e standardizzazione:")
    print(f"Train - Temporal: {X_temporal_train.shape}, Static: {X_static_train.shape}, Target: {y_train.shape}")
    print(f"Val - Temporal: {X_temporal_val.shape}, Static: {X_static_val.shape}, Target: {y_val.shape}")
    print(f"Test - Temporal: {X_temporal_test.shape}, Static: {X_static_test.shape}, Target: {y_test.shape}")

    # Prepara i dizionari di input
    train_data = {'temporal': X_temporal_train, 'static': X_static_train}
    val_data = {'temporal': X_temporal_val, 'static': X_static_val}
    test_data = {'temporal': X_temporal_test, 'static': X_static_test}

    joblib.dump(scaler_temporal, os.path.join(base_project_dir, f'{execute_name}_scaler_temporal.joblib'))
    joblib.dump(scaler_static, os.path.join(base_project_dir, f'{execute_name}_scaler_static.joblib'))
    joblib.dump(scaler_y, os.path.join(base_project_dir, f'{execute_name}_scaler_y.joblib'))

    return (train_data, y_train), (val_data, y_val), (test_data, y_test), (scaler_temporal, scaler_static, scaler_y)

In [ ]:
simulated_data = pd.read_parquet(f"{data_dir}simulated_data.parquet")
olive_varieties = pd.read_parquet(f"{data_dir}olive_varieties.parquet")

(train_data, train_targets), (val_data, val_targets), (test_data, test_targets), scalers = prepare_transformer_data(simulated_data, olive_varieties)

scaler_temporal, scaler_static, scaler_y = scalers

print("Temporal data shape:", train_data['temporal'].shape)
print("Static data shape:", train_data['static'].shape)
print("Target shape:", train_targets.shape)

In [ ]:
@keras.saving.register_keras_serializable()
class DataAugmentation(tf.keras.layers.Layer):
    """Custom layer per l'augmentation dei dati"""

    def __init__(self, noise_stddev=0.03, **kwargs):
        super().__init__(**kwargs)
        self.noise_stddev = noise_stddev

    def call(self, inputs, training=None):
        if training:
            return inputs + tf.random.normal(
                shape=tf.shape(inputs),
                mean=0.0,
                stddev=self.noise_stddev
            )
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"noise_stddev": self.noise_stddev})
        return config


@keras.saving.register_keras_serializable()
class PositionalEncoding(tf.keras.layers.Layer):
    """Custom layer per l'encoding posizionale"""

    def __init__(self, d_model, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model

    def build(self, input_shape):
        _, seq_length, _ = input_shape

        # Crea la matrice di encoding posizionale
        position = tf.range(seq_length, dtype=tf.float32)[:, tf.newaxis]
        div_term = tf.exp(
            tf.range(0, self.d_model, 2, dtype=tf.float32) *
            (-tf.math.log(10000.0) / self.d_model)
        )

        # Calcola sin e cos
        pos_encoding = tf.zeros((1, seq_length, self.d_model))
        pos_encoding_even = tf.sin(position * div_term)
        pos_encoding_odd = tf.cos(position * div_term)

        # Assegna i valori alle posizioni pari e dispari
        pos_encoding = tf.concat(
            [tf.expand_dims(pos_encoding_even, -1),
             tf.expand_dims(pos_encoding_odd, -1)],
            axis=-1
        )
        pos_encoding = tf.reshape(pos_encoding, (1, seq_length, -1))
        pos_encoding = pos_encoding[:, :, :self.d_model]

        # Salva l'encoding come peso non trainabile
        self.pos_encoding = self.add_weight(
            shape=(1, seq_length, self.d_model),
            initializer=tf.keras.initializers.Constant(pos_encoding),
            trainable=False,
            name='positional_encoding'
        )

        super().build(input_shape)

    def call(self, inputs):
        # Broadcast l'encoding posizionale sul batch
        batch_size = tf.shape(inputs)[0]
        pos_encoding_tiled = tf.tile(self.pos_encoding, [batch_size, 1, 1])
        return inputs + pos_encoding_tiled

    def get_config(self):
        config = super().get_config()
        config.update({"d_model": self.d_model})
        return config


@keras.saving.register_keras_serializable()
class WarmUpLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """Custom learning rate schedule with linear warmup and exponential decay."""

    def __init__(self, initial_learning_rate=1e-3, warmup_steps=500, decay_steps=5000):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = warmup_steps
        self.decay_steps = decay_steps

    def __call__(self, step):
        warmup_pct = tf.cast(step, tf.float32) / self.warmup_steps
        warmup_lr = self.initial_learning_rate * warmup_pct
        decay_factor = tf.pow(0.1, tf.cast(step, tf.float32) / self.decay_steps)
        decayed_lr = self.initial_learning_rate * decay_factor
        return tf.where(step < self.warmup_steps, warmup_lr, decayed_lr)

    def get_config(self):
        return {
            'initial_learning_rate': self.initial_learning_rate,
            'warmup_steps': self.warmup_steps,
            'decay_steps': self.decay_steps
        }


def create_olive_oil_transformer(temporal_shape, static_shape, num_outputs,
                                 d_model=128, num_heads=8, ff_dim=256,
                                 num_transformer_blocks=4, mlp_units=None,
                                 dropout=0.2):
    """
    Crea un transformer per la predizione della produzione di olio d'oliva.
    """
    # Input layers
    if mlp_units is None:
        mlp_units = [256, 128, 64]

    temporal_input = tf.keras.layers.Input(shape=temporal_shape, name='temporal')
    static_input = tf.keras.layers.Input(shape=static_shape, name='static')

    # === TEMPORAL PATH ===
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(temporal_input)
    x = DataAugmentation()(x)

    # Temporal projection
    x = tf.keras.layers.Dense(
        d_model // 2,
        activation='swish',
        kernel_regularizer=tf.keras.regularizers.l2(1e-5)
    )(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(
        d_model,
        activation='swish',
        kernel_regularizer=tf.keras.regularizers.l2(1e-5)
    )(x)

    # Positional encoding
    x = PositionalEncoding(d_model)(x)

    # Transformer blocks
    skip_connection = x
    for _ in range(num_transformer_blocks):
        # Self-attention
        attention_output = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model // num_heads,
            value_dim=d_model // num_heads
        )(x, x)
        attention_output = tf.keras.layers.Dropout(dropout)(attention_output)

        # Residual connection con pesi addestrabili
        residual_weights = tf.keras.layers.Dense(d_model, activation='sigmoid')(x)
        x = tfa.layers.StochasticDepth(survival_probability=0.3)([x, residual_weights * attention_output])
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

        # Feed-forward network
        ffn = tf.keras.layers.Dense(ff_dim, activation="swish")(x)
        ffn = tf.keras.layers.Dropout(dropout)(ffn)
        ffn = tf.keras.layers.Dense(d_model)(ffn)
        ffn = tf.keras.layers.Dropout(dropout)(ffn)

        # Second residual connection
        x = tfa.layers.StochasticDepth()([x, ffn])
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

    # Add final skip connection
    x = tfa.layers.StochasticDepth(survival_probability=0.5)([x, skip_connection])

    # Temporal pooling
    attention_pooled = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=d_model // 4
    )(x, x)
    attention_pooled = tf.keras.layers.GlobalAveragePooling1D()(attention_pooled)

    # Additional pooling operations
    avg_pooled = tf.keras.layers.GlobalAveragePooling1D()(x)
    max_pooled = tf.keras.layers.GlobalMaxPooling1D()(x)

    # Combine pooling results
    temporal_features = tf.keras.layers.Concatenate()(
        [attention_pooled, avg_pooled, max_pooled]
    )

    # === STATIC PATH ===
    static_features = tf.keras.layers.LayerNormalization(epsilon=1e-6)(static_input)
    for units in [256, 128, 64]:
        static_features = tf.keras.layers.Dense(
            units,
            activation='swish',
            kernel_regularizer=tf.keras.regularizers.l2(1e-5)
        )(static_features)
        static_features = tf.keras.layers.Dropout(dropout)(static_features)

    # === FEATURE FUSION ===
    combined = tf.keras.layers.Concatenate()([temporal_features, static_features])

    # === MLP HEAD ===
    x = combined
    for units in mlp_units:
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(
            units,
            activation="swish",
            kernel_regularizer=tf.keras.regularizers.l2(1e-5)
        )(x)
        x = tf.keras.layers.Dropout(dropout)(x)

    # Output layer
    outputs = tf.keras.layers.Dense(
        num_outputs,
        activation='linear',
        kernel_regularizer=tf.keras.regularizers.l2(1e-5)
    )(x)

    # Create model
    model = tf.keras.Model(
        inputs={'temporal': temporal_input, 'static': static_input},
        outputs=outputs,
        name='OilTransformer'
    )

    return model


def create_transformer_callbacks(target_names, val_data, val_targets):
    """
    Crea i callbacks per il training del modello.
    
    Parameters:
    -----------
    target_names : list
        Lista dei nomi dei target per il monitoraggio specifico
    val_data : dict
        Dati di validazione
    val_targets : array
        Target di validazione
    
    Returns:
    --------
    list
        Lista dei callbacks configurati
    """

    # Custom Metric per target specifici
    class TargetSpecificMetric(tf.keras.callbacks.Callback):
        def __init__(self, validation_data, target_names):
            super().__init__()
            self.validation_data = validation_data
            self.target_names = target_names

        def on_epoch_end(self, epoch, logs={}):
            x_val, y_val = self.validation_data
            y_pred = self.model.predict(x_val, verbose=0)

            for i, name in enumerate(self.target_names):
                mae = np.mean(np.abs(y_val[:, i] - y_pred[:, i]))
                logs[f'val_{name}_mae'] = mae


    callbacks = [
        # Early Stopping
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            min_delta=0.0005,
            mode='min'
        ),

        # Model Checkpoint
        tf.keras.callbacks.ModelCheckpoint(
            filepath=f'{execute_name}_best_oil_model.h5',
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            save_weights_only=True
        ),

        # Metric per target specifici
        TargetSpecificMetric(
            validation_data=(val_data, val_targets),
            target_names=target_names
        ),

        # Reduce LR on Plateau
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-6,
            verbose=1
        ),

        # TensorBoard logging
        tf.keras.callbacks.TensorBoard(
            log_dir=f'./logs_{execute_name}',
            histogram_freq=1,
            write_graph=True,
            update_freq='epoch'
        )
    ]

    return callbacks


def compile_model(model, learning_rate=1e-3):
    """
    Compila il modello con le impostazioni standard.
    """
    lr_schedule = WarmUpLearningRateSchedule(
        initial_learning_rate=learning_rate,
        warmup_steps=500,
        decay_steps=5000
    )

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(
            learning_rate=lr_schedule,
            weight_decay=0.01
        ),
        loss=tf.keras.losses.Huber(),
        metrics=['mae']
    )

    return model


def setup_transformer_training(train_data, train_targets, val_data, val_targets):
    """
    Configura e prepara il transformer con dimensioni dinamiche basate sui dati.
    """
    # Estrai le shape dai dati
    temporal_shape = (train_data['temporal'].shape[1], train_data['temporal'].shape[2])
    static_shape = (train_data['static'].shape[1],)
    num_outputs = train_targets.shape[1]

    print(f"Shape rilevate:")
    print(f"- Temporal shape: {temporal_shape}")
    print(f"- Static shape: {static_shape}")
    print(f"- Numero di output: {num_outputs}")

    # Target names basati sul numero di output
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']

    # Assicurati che il numero di target names corrisponda al numero di output
    assert len(target_names) == num_outputs, \
        f"Il numero di target names ({len(target_names)}) non corrisponde al numero di output ({num_outputs})"

    # Crea il modello con le dimensioni rilevate
    model = create_olive_oil_transformer(
        temporal_shape=temporal_shape,
        static_shape=static_shape,
        num_outputs=num_outputs
    )

    # Compila il modello
    model = compile_model(model)

    # Crea i callbacks
    callbacks = create_transformer_callbacks(target_names, val_data, val_targets)

    return model, callbacks, target_names


def train_transformer(train_data, train_targets, val_data, val_targets, epochs=150, batch_size=64, save_name='final_model'):
    """
    Funzione principale per l'addestramento del transformer con ottimizzazioni.
    """
    # Conversione dei dati in tf.data.Dataset per una gestione più efficiente della memoria
    train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_targets))\
        .cache()\
        .shuffle(buffer_size=1024)\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_targets))\
        .cache()\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)

    # Setup del modello
    strategy = tf.distribute.MirroredStrategy() if len(tf.config.list_physical_devices('GPU')) > 1 else tf.distribute.get_strategy()
    
    with strategy.scope():
        model, callbacks, target_names = setup_transformer_training(
            train_data, train_targets, val_data, val_targets
        )

    # Mostra il summary del modello
    model.summary()
    
    try:
        keras.utils.plot_model(model, f"{execute_name}_{save_name}.png", show_shapes=True)
    except Exception as e:
        print(f"Warning: Could not create model plot: {e}")

    # Training con gestione degli errori
    try:
        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1,
            workers=4,
            use_multiprocessing=True
        )
    except tf.errors.ResourceExhaustedError:
        print("Memoria GPU esaurita, riprovo con batch size più piccolo...")
        # Riprova con batch size più piccolo
        batch_size = batch_size // 2
        train_dataset = train_dataset.unbatch().batch(batch_size)
        val_dataset = val_dataset.unbatch().batch(batch_size)
        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

    # Salva il modello finale
    try:
        save_path = f'{execute_name}_{save_name}.keras'
        model.save(save_path, save_format='keras')
        
        os.makedirs(f'{execute_name}/weights', exist_ok=True)
        model.save_weights(f'{execute_name}/weights')
        print(f"\nModello salvato in: {save_path}")
    except Exception as e:
        print(f"Warning: Could not save model: {e}")

    return model, history

In [ ]:
model, history = train_transformer(train_data, train_targets, val_data, val_targets, 150, 256)

In [ ]:
percentage_errors, absolute_errors = calculate_real_error(model, val_data, val_targets, scaler_y)

In [ ]:
def evaluate_model_performance(model, data, targets, set_name=""):
    """
    Valuta le performance del modello su un set di dati specifico.
    """
    predictions = model.predict(data, verbose=0)

    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']
    metrics = {}

    for i, name in enumerate(target_names):
        mae = np.mean(np.abs(targets[:, i] - predictions[:, i]))
        mse = np.mean(np.square(targets[:, i] - predictions[:, i]))
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((targets[:, i] - predictions[:, i]) / (targets[:, i] + 1e-7))) * 100

        metrics[f"{name}_mae"] = mae
        metrics[f"{name}_rmse"] = rmse
        metrics[f"{name}_mape"] = mape

    if set_name:
        print(f"\nPerformance sul set {set_name}:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

    return metrics


def retrain_model(base_model, train_data, train_targets,
                  val_data, val_targets,
                  test_data, test_targets,
                  epochs=50, batch_size=128):
    """
    Implementa il retraining del modello con i dati combinati.
    """
    print("Valutazione performance iniziali del modello...")
    initial_metrics = {
        'train': evaluate_model_performance(base_model, train_data, train_targets, "training"),
        'val': evaluate_model_performance(base_model, val_data, val_targets, "validazione"),
        'test': evaluate_model_performance(base_model, test_data, test_targets, "test")
    }

    # Combina i dati per il retraining
    combined_data = {
        'temporal': np.concatenate([train_data['temporal'], val_data['temporal'], test_data['temporal']]),
        'static': np.concatenate([train_data['static'], val_data['static'], test_data['static']])
    }
    combined_targets = np.concatenate([train_targets, val_targets, test_targets])

    # Crea una nuova suddivisione per la validazione
    indices = np.arange(len(combined_targets))
    np.random.shuffle(indices)

    split_idx = int(len(indices) * 0.9)
    train_idx, val_idx = indices[:split_idx], indices[split_idx:]

    # Prepara i dati per il retraining
    retrain_data = {k: v[train_idx] for k, v in combined_data.items()}
    retrain_targets = combined_targets[train_idx]
    retrain_val_data = {k: v[val_idx] for k, v in combined_data.items()}
    retrain_val_targets = combined_targets[val_idx]

    # Configura callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            min_delta=0.0001
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=f'{execute_name}_retrained_best_oil_model.h5',
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            save_weights_only=True
        )
    ]

    # Imposta learning rate per il fine-tuning
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-4,
            decay_steps=1000,
            decay_rate=0.9
        ),
        weight_decay=0.01
    )

    # Ricompila il modello con il nuovo optimizer
    base_model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.Huber(),
        metrics=['mae']
    )

    print("\nAvvio retraining...")
    history = base_model.fit(
        retrain_data,
        retrain_targets,
        validation_data=(retrain_val_data, retrain_val_targets),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

    print("\nValutazione performance finali...")
    final_metrics = {
        'train': evaluate_model_performance(base_model, train_data, train_targets, "training"),
        'val': evaluate_model_performance(base_model, val_data, val_targets, "validazione"),
        'test': evaluate_model_performance(base_model, test_data, test_targets, "test")
    }

    # Salva il modello finale
    save_path = f'{execute_name}_retrained_model.keras'
    os.makedirs(f'{execute_name}_retrained/weights', exist_ok=True)
    
    base_model.save_weights(f'{execute_name}_retrained/weights')
    base_model.save(save_path, save_format='keras')
    print(f"\nModello riaddestrato salvato in: {save_path}")

    # Report miglioramenti
    print("\nMiglioramenti delle performance:")
    for dataset in ['train', 'val', 'test']:
        print(f"\nSet {dataset}:")
        for metric in initial_metrics[dataset].keys():
            initial = initial_metrics[dataset][metric]
            final = final_metrics[dataset][metric]
            improvement = ((initial - final) / initial) * 100
            print(f"{metric}: {improvement:.2f}% di miglioramento")

    return base_model, history, final_metrics


def start_retraining(model_path, train_data, train_targets,
                     val_data, val_targets,
                     test_data, test_targets,
                     epochs=50, batch_size=128):
    """
    Avvia il processo di retraining in modo sicuro.
    """
    try:
        print("Caricamento del modello...")
        base_model = tf.keras.models.load_model(model_path, compile=False)
        print("Modello caricato con successo!")

        return retrain_model(
            base_model=base_model,
            train_data=train_data,
            train_targets=train_targets,
            val_data=val_data,
            val_targets=val_targets,
            test_data=test_data,
            test_targets=test_targets,
            epochs=epochs,
            batch_size=batch_size
        )
    except Exception as e:
        print(f"Errore durante il retraining: {str(e)}")
        raise

In [ ]:
model_path = f'{execute_name}_final_model.keras'

retrained_model, retrain_history, final_metrics = start_retraining(
    model_path=model_path,
    train_data=train_data,
    train_targets=train_targets,
    val_data=val_data,
    val_targets=val_targets,
    test_data=test_data,
    test_targets=test_targets,
    epochs=50,
    batch_size=128
)

In [ ]:
percentage_errors, absolute_errors = calculate_real_error(retrained_model, val_data, val_targets, scaler_y)

In [ ]:
class ProbabilityFunctions:
    """
    Classe per calcolare e visualizzare PMF e CMF usando TensorFlow.
    """
    
    @staticmethod
    def calculate_pmf(data, bins=50, range_limits=None):
        """
        Calcola la PMF (Probability Mass Function) usando TensorFlow.
        
        Args:
            data: Tensor dei dati
            bins: Numero di bin per l'istogramma
            range_limits: Tuple (min, max) per i limiti dell'istogramma
            
        Returns:
            bin_edges: Bordi dei bin
            pmf: Valori della PMF normalizzati
        """
        # Converti i dati in tensor se non lo sono già
        if not isinstance(data, tf.Tensor):
            data = tf.convert_to_tensor(data, dtype=tf.float32)
            
        # Calcola i limiti se non specificati
        if range_limits is None:
            range_limits = (tf.reduce_min(data), tf.reduce_max(data))
            
        # Calcola l'istogramma
        counts = tf.histogram_fixed_width(data, range_limits, bins)
        
        # Normalizza per ottenere la PMF
        pmf = tf.cast(counts, tf.float32) / tf.reduce_sum(tf.cast(counts, tf.float32))
        
        # Calcola i bordi dei bin
        bin_width = (range_limits[1] - range_limits[0]) / bins
        bin_edges = tf.range(range_limits[0], range_limits[1] + bin_width, bin_width)
        
        return bin_edges.numpy(), pmf.numpy()
    
    @staticmethod
    def calculate_cmf(pmf):
        """
        Calcola la CMF (Cumulative Mass Function) dalla PMF.
        
        Args:
            pmf: Array della PMF
            
        Returns:
            cmf: Array della CMF
        """
        # Converti in tensor se non lo è già
        if not isinstance(pmf, tf.Tensor):
            pmf = tf.convert_to_tensor(pmf, dtype=tf.float32)
            
        # Calcola la CMF usando cumsum
        cmf = tf.cumsum(pmf)
        
        return cmf.numpy()
    
    @staticmethod
    def calculate_statistics(data):
        """
        Calcola statistiche di base usando TensorFlow.
        
        Args:
            data: Tensor dei dati
            
        Returns:
            dict: Dizionario con le statistiche
        """
        if not isinstance(data, tf.Tensor):
            data = tf.convert_to_tensor(data, dtype=tf.float32)
            
        mean = tf.reduce_mean(data)
        variance = tf.reduce_variance(data)
        std = tf.sqrt(variance)
        
        return {
            'mean': mean.numpy(),
            'variance': variance.numpy(),
            'std': std.numpy(),
            'min': tf.reduce_min(data).numpy(),
            'max': tf.reduce_max(data).numpy(),
            'median': tf.raw_ops.TopKV2(input=data, k=tf.size(data)//2)[0][-1].numpy()
        }
    
    @staticmethod
    def plot_distributions(bin_edges, pmf, cmf, title="Probability and Cumulative Distributions"):
        """
        Visualizza PMF e CMF.
        
        Args:
            bin_edges: Bordi dei bin
            pmf: Array della PMF
            cmf: Array della CMF
            title: Titolo del plot
        """
        import matplotlib.pyplot as plt
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        
        # Plot PMF
        ax1.bar(bin_edges[:-1], pmf, width=np.diff(bin_edges), alpha=0.5, 
                align='edge', label='PMF')
        ax1.set_title('Probability Mass Function')
        ax1.set_ylabel('Probability')
        ax1.grid(True)
        ax1.legend()
        
        # Plot CMF
        ax2.plot(bin_edges[:-1], cmf, 'r-', label='CMF')
        ax2.set_title('Cumulative Mass Function')
        ax2.set_ylabel('Cumulative Probability')
        ax2.set_xlabel('Value')
        ax2.grid(True)
        ax2.legend()
        
        plt.suptitle(title)
        plt.tight_layout()
        plt.show()

# Esempio di utilizzo
def demonstrate_probability_functions():
    """
    Dimostra l'uso delle funzioni di probabilità.
    """
    # Genera dati di esempio
    tf.random.set_seed(42)
    data = tf.concat([
        tf.random.normal([1000], mean=0, stddev=1),
        tf.random.normal([500], mean=3, stddev=0.5)
    ], axis=0)
    
    # Inizializza la classe
    prob = ProbabilityFunctions()
    
    # Calcola PMF e CMF
    bin_edges, pmf = prob.calculate_pmf(data, bins=50)
    cmf = prob.calculate_cmf(pmf)
    
    # Calcola statistiche
    stats = prob.calculate_statistics(data)
    
    print("Statistiche di Base:")
    for key, value in stats.items():
        print(f"{key}: {value:.3f}")
    
    # Visualizza le distribuzioni
    prob.plot_distributions(bin_edges, pmf, cmf, "Distribuzioni di Probabilità")

In [ ]:
def analyze_model_predictions(model, test_data, test_targets, scaler_y):
    """
    Analizza le distribuzioni di probabilità delle predizioni del modello.
    
    Args:
        model: Modello TensorFlow addestrato
        test_data: Dati di test
        test_targets: Target di test
        scaler_y: Scaler usato per denormalizzare i target
    """
    # Ottieni le predizioni
    predictions = model.predict(test_data)
    
    # Denormalizza predizioni e target
    predictions_real = scaler_y.inverse_transform(predictions)
    targets_real = scaler_y.inverse_transform(test_targets)
    
    # Inizializza la classe per l'analisi delle probabilità
    prob = ProbabilityFunctions()
    
    # Analizza ogni target
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']
    
    for i, target in enumerate(target_names):
        print(f"\nAnalisi per {target}")
        print("-" * 50)
        
        # Calcola errori
        errors = predictions_real[:, i] - targets_real[:, i]
        
        # Calcola statistiche degli errori
        error_stats = prob.calculate_statistics(errors)
        print("\nStatistiche degli Errori:")
        for key, value in error_stats.items():
            print(f"{key}: {value:.3f}")
        
        # Calcola PMF e CMF degli errori
        bin_edges, pmf = prob.calculate_pmf(errors, bins=50)
        cmf = prob.calculate_cmf(pmf)
        
        # Visualizza le distribuzioni degli errori
        prob.plot_distributions(bin_edges, pmf, cmf, 
                              f"Distribuzione degli Errori - {target}")
        
        # Calcola intervalli di confidenza
        confidence_levels = [0.68, 0.95, 0.99]  # 1σ, 2σ, 3σ
        for level in confidence_levels:
            lower_idx = np.searchsorted(cmf, (1 - level) / 2)
            upper_idx = np.searchsorted(cmf, (1 + level) / 2)
            
            print(f"\nIntervallo di Confidenza {level*100}%:")
            print(f"Range: [{bin_edges[lower_idx]:.2f}, {bin_edges[upper_idx]:.2f}]")
            
def analyze_feature_importance(model, test_data, feature_names):
    """
    Analizza l'importanza delle feature usando perturbazione.
    
    Args:
        model: Modello TensorFlow addestrato
        test_data: Dati di test
        feature_names: Lista dei nomi delle feature
    """
    base_prediction = model.predict(test_data)
    feature_importance = {}
    
    # Per ogni feature
    for i, feature in enumerate(feature_names):
        # Crea copia perturbata dei dati
        perturbed_data = test_data.copy()
        noise = tf.random.normal(shape=perturbed_data[feature].shape,
                               mean=0, stddev=0.1)
        perturbed_data[feature] = perturbed_data[feature] + noise
        
        # Calcola nuova predizione
        perturbed_prediction = model.predict(perturbed_data)
        
        # Calcola impatto della perturbazione
        impact = tf.reduce_mean(tf.abs(perturbed_prediction - base_prediction))
        feature_importance[feature] = impact.numpy()
    
    # Normalizza e ordina le importanze
    total_importance = sum(feature_importance.values())
    feature_importance = {k: v/total_importance for k, v in feature_importance.items()}
    feature_importance = dict(sorted(feature_importance.items(), 
                                   key=lambda x: x[1], reverse=True))
    
    return feature_importance

In [ ]:
def run_comprehensive_analysis(retrained_model, test_data, test_targets, scaler_y):
    """
    Esegue un'analisi completa del modello includendo errori,
    importanza delle feature e distribuzioni.
    """
    print("=== ANALISI COMPLETA DEL MODELLO ===")
    
    # 1. Analisi degli errori
    print("\n1. ANALISI DEGLI ERRORI")
    print("-" * 50)
    analyze_model_predictions(retrained_model, test_data, test_targets, scaler_y)
    
    # 2. Analisi dell'importanza delle feature
    print("\n2. IMPORTANZA DELLE FEATURE")
    print("-" * 50)
    
    # Definisci i nomi delle feature
    temporal_features = ['temp_mean', 'precip_sum', 'solar_energy_sum']
    static_features = ['ha']  # Aggiungi le tue feature statiche
    
    all_features = temporal_features + static_features
    importance = analyze_feature_importance(retrained_model, test_data, all_features)
    
    print("\nImportanza relativa delle feature:")
    for feature, imp in sorted(importance.items(), key=lambda x: x[1], reverse=True):
        print(f"{feature}: {imp:.4f}")
        
    # 3. Analisi distribuzionale
    print("\n3. ANALISI DISTRIBUZIONALE")
    print("-" * 50)
    
    prob = ProbabilityFunctions()
    predictions = retrained_model.predict(test_data)
    predictions_real = scaler_y.inverse_transform(predictions)
    targets_real = scaler_y.inverse_transform(test_targets)
    
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 
                    'avg_oil_prod', 'total_water_need']
    
    for i, target in enumerate(target_names):
        print(f"\nAnalisi distribuzionale per {target}")
        
        # Analizza predizioni
        bin_edges_pred, pmf_pred = prob.calculate_pmf(predictions_real[:, i])
        cmf_pred = prob.calculate_cmf(pmf_pred)
        
        # Analizza target reali
        bin_edges_true, pmf_true = prob.calculate_pmf(targets_real[:, i])
        cmf_true = prob.calculate_cmf(pmf_true)
        
        # Statistiche
        stats_pred = prob.calculate_statistics(predictions_real[:, i])
        stats_true = prob.calculate_statistics(targets_real[:, i])
        
        print("\nStatistiche Predizioni:")
        for key, value in stats_pred.items():
            print(f"{key}: {value:.3f}")
            
        print("\nStatistiche Target Reali:")
        for key, value in stats_true.items():
            print(f"{key}: {value:.3f}")
        
        # Visualizza distribuzioni
        prob.plot_distributions(bin_edges_pred, pmf_pred, cmf_pred,
                              f"Distribuzione Predizioni - {target}")
        prob.plot_distributions(bin_edges_true, pmf_true, cmf_true,
                              f"Distribuzione Target Reali - {target}")

In [ ]:
run_comprehensive_analysis(retrained_model, test_data, test_targets, scaler_y)